# exploring 

In [22]:
import pandas as pd
import numpy as np
import datetime
import re
from scipy import stats

In [131]:
pd.set_option("display.max_rows", 300)

In [23]:
df = pd.read_csv("../Cleaned_Data/Tasks.csv")

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25012 entries, 0 to 25011
Data columns (total 11 columns):
Unnamed: 0          25012 non-null int64
User                25012 non-null object
Project             25012 non-null object
Task_Category       25012 non-null object
Task_Description    25012 non-null object
Date                25012 non-null object
Duration            25012 non-null object
Time_Decimal        25012 non-null float64
Cost_per_hour       25012 non-null float64
Position            25012 non-null object
Task_Cost           25012 non-null float64
dtypes: float64(3), int64(1), object(7)
memory usage: 2.1+ MB


In [25]:
# reording the data to make it more understandable
df.Cost_per_hour = df.Cost_per_hour.round(2)
df.Time_Decimal = df.Time_Decimal.round(2)
df.drop(columns=["Unnamed: 0"], inplace=True)

In [26]:
 df = df[["Date", "Duration", "Project", "Task_Category", "Task_Cost", "Task_Description", "User", "Position", "Cost_per_hour","Time_Decimal"]]

In [27]:
df

,Date,Duration,Project,Task_Category,Task_Cost,Task_Description,User,Position,Cost_per_hour,Time_Decimal
0,2017-09-01,01:17:00,Revenify,Development,26.41,Feedback on budgets index PR and Standup meeti...,UI13,Senior Back End Developer,20.58,1.28
1,2017-09-01,02:30:00,WWU,Development,62.50,Create graphs in XLSX format from user input,UI36,Senior Full-Stack Developer,25.00,2.50
2,2017-09-01,00:36:49,WWU,Development,15.34,Create graphs in XLSX format from user input,UI36,Senior Full-Stack Developer,25.00,0.61
3,2017-09-01,00:12:05,WWU,Development,5.03,Weekly Standup,UI38,Project Manager / Tech Lead,25.00,0.20
4,2017-09-01,00:09:07,WWU,Development,3.80,Weekly standup,UI36,Senior Full-Stack Developer,25.00,0.15
...,...,...,...,...,...,...,...,...,...,...
25007,2019-09-28,00:41:22,CareBetter,Development,24.13,[DB_ADMIN_002] Create user <> admin association,UI35,Senior Full-Stack Developer,35.00,0.69
25008,2019-09-28,00:20:55,CandidatePool,Development,8.46,Working on adding email and owner to applicant...,UI20,Back End Developer,24.26,0.35
25009,2019-09-28,01:02:00,CandidatePool,Development,29.02,[FrontEnd] : Add an Alert when The User does n...,UI14,Front End Developer,28.08,1.03
25010,2019-09-28,00:20:59,CareBetter,Development,12.24,[DB_ADMIN_002] Create user <> admin association,UI35,Senior Full-Stack Developer,35.00,0.35


In [28]:
df.columns

Index(['Date', 'Duration', 'Project', 'Task_Category', 'Task_Cost',
       'Task_Description', 'User', 'Position', 'Cost_per_hour',
       'Time_Decimal'],
      dtype='object')

In [29]:
# 24 projects! are massured
len(list(df.Project.unique()))

24

In [30]:
# turing the Duration into timedelata
df.Duration = pd.to_timedelta(df.Duration)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25012 entries, 0 to 25011
Data columns (total 10 columns):
Date                25012 non-null object
Duration            25012 non-null timedelta64[ns]
Project             25012 non-null object
Task_Category       25012 non-null object
Task_Cost           25012 non-null float64
Task_Description    25012 non-null object
User                25012 non-null object
Position            25012 non-null object
Cost_per_hour       25012 non-null float64
Time_Decimal        25012 non-null float64
dtypes: float64(3), object(6), timedelta64[ns](1)
memory usage: 1.9+ MB


# Avg cost:
1) by project
2) By catogory (Devlopment (Backend, Frontend, Full stack, Tech Lead) 
3) by seniorty 
4) calulate 


## Avg 
### all
median = 32.16 (from all projects)
mean = 32.13 (from all hours)
as we can see 32.1 USD is the avg of a programmer 
### over 1000 hours  

### under 1000 hours but more thne 100

median = 34.42 

In [32]:
# call the mean
df.Task_Cost.sum()/df.Time_Decimal.sum()

32.136385297567145

In [33]:
#cal the median per 
# to most expensive per hour project
exp_pro = df[['Project', 'Task_Category', 'Task_Cost', 'Duration', 'Time_Decimal']] 

In [34]:
cost_per_hour_per_pro = (exp_pro.groupby(by="Project").sum()
.assign(mean_cost_per_hour_per_pro= lambda x : round(x.Task_Cost / x.Time_Decimal, 2)))

In [35]:
cost_per_hour_per_pro.describe()


,Task_Cost,Time_Decimal,mean_cost_per_hour_per_pro
count,24.000000,24.000000,24.000000
mean,63923.702500,1989.137917,34.075833
std,95160.509228,3380.014861,10.289358
min,392.000000,13.970000,18.000000
25%,7706.195000,207.205000,26.637500
50%,22601.150000,710.495000,32.165000
75%,60723.685000,1766.385000,39.920000
max,365083.030000,14963.730000,64.600000


In [36]:
# lets take projects out are over 1000 hours  
len(cost_per_hour_per_pro[cost_per_hour_per_pro.Time_Decimal > 1000])
# it is 10 project!
# meadian is 34.42

10

In [37]:
list(cost_per_hour_per_pro[cost_per_hour_per_pro.Time_Decimal > 1000].index)

['AgeSolvency',
 'CandidatePool',
 'Credious',
 'Finizon',
 'GDP',
 'NestPride',
 'PilotAxis',
 'Revenify',
 'TeachZip',
 'WeVape']

In [38]:
cost_per_hour_per_pro[cost_per_hour_per_pro.Time_Decimal > 1000].describe()

,Task_Cost,Time_Decimal,mean_cost_per_hour_per_pro
count,10.000000,10.000000,10.000000
mean,137911.711000,4325.511000,33.740000
std,112165.047036,4322.844214,7.031794
min,33014.990000,1178.090000,24.400000
25%,45966.075000,1488.270000,28.257500
50%,103270.190000,2670.550000,34.420000
75%,218455.507500,5271.385000,39.517500
max,365083.030000,14963.730000,42.040000


In [39]:
cost_per_hour_per_pro[cost_per_hour_per_pro.Time_Decimal > 1000]

,Task_Cost,Time_Decimal,mean_cost_per_hour_per_pro
Project,,,
AgeSolvency,163039.50,3878.65,42.04
CandidatePool,56206.17,1414.82,39.73
Credious,365083.03,14963.73,24.40
Finizon,238705.00,5735.63,41.62
GDP,74276.23,1939.68,38.29
NestPride,37047.82,1278.65,28.97
PilotAxis,132264.15,3401.42,38.88
Revenify,42552.71,1708.62,24.90
TeachZip,236927.51,7755.82,30.55


In [40]:
# lets take projects out are under 1000 hours  and over 100 hours
len(cost_per_hour_per_pro[(cost_per_hour_per_pro.Time_Decimal < 1000) & (cost_per_hour_per_pro.Time_Decimal > 100)])
# it is 11 projects

11

In [41]:
cost_per_hour_per_pro[(cost_per_hour_per_pro.Time_Decimal < 1000) & (cost_per_hour_per_pro.Time_Decimal > 100)].describe()

,Task_Cost,Time_Decimal,mean_cost_per_hour_per_pro
count,11.000000,11.000000,11.000000
mean,13728.880909,392.538182,36.827273
std,9169.777765,270.749075,12.835255
min,2536.420000,106.030000,18.000000
25%,7473.670000,198.460000,27.265000
50%,11944.460000,305.350000,37.630000
75%,17697.370000,525.440000,40.905000
max,33001.500000,893.570000,64.600000


# tasks from project


In [154]:
def tasks_with_word(word):
    return df[df.Task_Description.str.lower().str.contains(word, regex=True)]

In [153]:
def tasks_with_list(lst):
    pattern='|'.join(lst)
    return df[df.Task_Description.str.lower().str.contains(pattern, regex=True)]

In [50]:
df[df.Task_Description.str.contains("[mM]eeting|[Ww]eekly|[Dd]aily|[Ss]tandup|[Cc]all|[dD]iscussion", regex=True) & (df.Time_Decimal > 1) & (df.Time_Decimal < 5)]

,Date,Duration,Project,Task_Category,Task_Cost,Task_Description,User,Position,Cost_per_hour,Time_Decimal
0,2017-09-01,01:17:00,Revenify,Development,26.41,Feedback on budgets index PR and Standup meeti...,UI13,Senior Back End Developer,20.58,1.28
56,2017-09-08,03:17:10,Revenify,Development,67.63,Standup meeting and working on transunion conn...,UI13,Senior Back End Developer,20.58,3.29
90,2017-09-13,01:03:53,Revenify,Development,21.91,"Raising PR for Transunion integration, feedbac...",UI13,Senior Back End Developer,20.58,1.06
113,2017-09-15,01:09:57,Revenify,Development,23.99,Encrypt sensitive information in the database ...,UI13,Senior Back End Developer,20.58,1.17
136,2017-09-18,03:53:36,Revenify,Development,80.13,Resolving PR conflicts and Standup meeting,UI13,Senior Back End Developer,20.58,3.89
...,...,...,...,...,...,...,...,...,...,...
24819,2019-09-19,02:30:00,LearnSage,Development,132.25,office meeting,UI04,Senior Front End Developer,52.90,2.50
24821,2019-09-19,03:40:57,CandidatePool,Development,221.12,"XXX : Testing Latest Changes, Working with Tea...",UI03,Senior Back End Developer,60.05,3.68
24890,2019-09-23,03:00:00,LearnSage,Development,158.70,"office meeting, modal gallery - full screen wh...",UI04,Senior Front End Developer,52.90,3.00
24909,2019-09-24,04:00:41,SharpDev,Development,258.08,"Github - CircleCI Integration, call with Devop...",UI32,Project Manager / Tech Lead,64.34,4.01


In [161]:
stop_word = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now", 'xxx', 'x', '-', 'new', 'fafsa', '+', '&', '[fafsa]',]
stop_word1 = ['working', 'work', 'user', 'experience', 'sf', 'continue', ':', 'one', '2.', 'agent', 'able', 'credit','end', 'prospect', 'status', 'based',  'endpoint', 'management', '','number', 'docker', 'text', 'back', 'component', 'version',  'prs', 'bds', 'need',  'portal','website', 'plaid', 'code', 'add', 
 '1.', 'xxx', 'users', '[fafsa]',  'go',  '[xxx', 'form', 'tab',  'profile',  'site', 'job', 'tu' ,'debt', 'last', 'crm',  'landing',  '/',  'button', 'financial',  'secxty', 'gds', 'tradeline', 'screen', 'app', 'pages', 'admin', 'make', 'xxx,', 'page', 'manager']
stop_word = stop_word + stop_word1

In [162]:
word_ser = pd.Series([re.sub(r'[^a-zA-Z0-9]','', i)for i in' '.join(df.Task_Description).lower().split()]).value_counts()
word_ser

xxx                    7490
on                     5825
                       4868
to                     4857
the                    4677
                       ... 
anable                    1
dhakad                    1
increment                 1
bimonthlypaymentday       1
tunnel                    1
Length: 6342, dtype: int64

In [163]:
testing = ['test ', 'testing', 'tested', 'qa ',    ]
meetings = ['meeting', 'weekly', 'daily', 'standup', 'call' , 'discussion', 'talk', 'sync ', 'demo ']
planing = ['sprint', 'plan']
code_review = ['review']
deployment = ['staging', 'production', 'env ', 'environment', 'pr ', "prod ", 'deploy','releas',  ]
seting_up = ['setup','setting', 'set ', ]
automation = ['automation',]
fixing_bugs = ['fix ', 'fixed', 'fixing', 'fixes', 'refactor', 'bug', 'solve', 'solving', 'issue', 'error']
research = ['research']


In [164]:
lst_all_words = ' '.join(df.Task_Description).lower().split()
lst_all_words_cleaned = [re.sub(r'[^a-zA-Z]','', i)for i in lst_all_words]
lst_all_words_cleaned_no_stop_words = [word for word in lst_all_words_cleaned if word not in stop_word]
list(pd.Series(lst_all_words_cleaned_no_stop_words).value_counts()[:200].index)

['testing',
 'review',
 'meeting',
 'staging',
 'standup',
 'automation',
 'sprint',
 'production',
 'update',
 'changes',
 'issue',
 'weekly',
 'create',
 'planning',
 'deployment',
 'test',
 'fix',
 'design',
 'frontend',
 'call',
 'discussion',
 'fixing',
 'daily',
 'backend',
 'integration',
 'server',
 'release',
 'funnel',
 'data',
 'flow',
 'deploy',
 'issues',
 'adding',
 'team',
 'bug',
 'tests',
 'debts',
 'setup',
 'qa',
 'client',
 'feature',
 'project',
 'implement',
 'creating',
 'fixes',
 'program',
 'research',
 'env',
 'pr',
 'devops',
 'api',
 'tested',
 'report',
 'error',
 'refactor',
 'implementation',
 'information',
 'tickets',
 'talk',
 'change',
 'updating',
 'sanity',
 'view',
 'functionality',
 'login',
 'prod',
 'db',
 'feedback',
 'documentation',
 'tradelines',
 'details',
 'search',
 'deploying',
 'tasks',
 'table',
 'setting',
 'validation',
 'submitter',
 'checking',
 'cases',
 'support',
 'adjustments',
 'field',
 'summary',
 'dashboard',
 'logic',
 'u

In [218]:
tasks_with_word('data')

,Date,Duration,Project,Task_Category,Task_Cost,Task_Description,User,Position,Cost_per_hour,Time_Decimal
0,2017-09-01,01:17:00,Revenify,Development,26.41,Feedback on budgets index PR and Standup meeti...,UI13,Senior Back End Developer,20.58,1.28
7,2017-09-02,01:04:52,WWU,Development,27.03,Do not allow data analysis to occur while the ...,UI36,Senior Full-Stack Developer,25.00,1.08
15,2017-09-04,01:41:00,WWU,Development,42.08,Translate Excel data to Hebrew,UI36,Senior Full-Stack Developer,25.00,1.68
21,2017-09-04,02:17:47,Revenify,Development,47.26,A user should only see budget data 90 days back,UI13,Senior Back End Developer,20.58,2.30
23,2017-09-05,05:16:39,Revenify,Development,108.61,Doing some tests around the logic of a user s...,UI13,Senior Back End Developer,20.58,5.28
...,...,...,...,...,...,...,...,...,...,...
24846,2019-09-20,02:05:00,CandidatePool,Development,125.10,XXX : Creating Data in Local Server for Testin...,UI03,Senior Back End Developer,60.05,2.08
24849,2019-09-20,01:25:42,Finizon,Development,77.60,Secxty and Data Integrity review,UI45,Project Manager / Tech Lead,54.33,1.43
24860,2019-09-22,02:08:14,LearnSage,Development,113.06,using google scripts api to load data,UI04,Senior Front End Developer,52.90,2.14
24891,2019-09-23,04:11:49,CandidatePool,Development,252.01,XXX : Testing Latest PRs + Injecting Data to S...,UI03,Senior Back End Developer,60.05,4.20


In [183]:
def idea(df):
    lst_all_words = ' '.join(df.Task_Description).lower().split()
    lst_all_words_cleaned = [re.sub(r'[^a-zA-Z]','', i)for i in lst_all_words]
    lst_all_words_cleaned_no_stop_words = [word for word in lst_all_words_cleaned if word not in stop_word]
    return pd.Series(lst_all_words_cleaned_no_stop_words).value_counts()

In [204]:
idea(tasks_with_word(''demo ))

validation                                         208
validations                                         57
error                                               36
fix                                                 34
added                                               22
                                                  ... 
bbxxx                                                1
parentinformationpersonalinformationotherparent      1
signature                                            1
dashes                                               1
autosuggest                                          1
Length: 664, dtype: int64

In [217]:
tasks_with_list(["demo "])

,Date,Duration,Project,Task_Category,Task_Cost,Task_Description,User,Position,Cost_per_hour,Time_Decimal
176,2017-09-22,04:41:56,Revenify,Development,96.71,"TransUnion demo postman requests, Standup Meet...",UI13,Senior Back End Developer,20.58,4.70
187,2017-09-25,01:26:19,Revenify,Development,29.61,TransUnion demo setting requests and stand up ...,UI13,Senior Back End Developer,20.58,1.44
253,2017-10-02,03:47:40,Revenify,Development,82.04,Working on Plaid demo and update TU demo endpoint,UI13,Senior Back End Developer,21.62,3.79
301,2017-10-09,00:33:00,Credious,Development,15.12,Plaid Demo Setup,UI38,Project Manager / Tech Lead,27.50,0.55
307,2017-10-10,00:40:22,Revenify,Development,14.55,Plaid Demo with FE team,UI13,Senior Back End Developer,21.62,0.67
318,2017-10-10,00:32:44,Credious,Development,15.00,Plaid Demo meeting,UI38,Project Manager / Tech Lead,27.50,0.55
632,2017-11-07,05:54:06,Revenify,Development,115.93,Working on CRM support for first_name and last...,UI13,Senior Back End Developer,19.64,5.90
649,2017-11-08,03:14:13,Revenify,Development,63.59,Call and demo with XXX to go over PRs and feed...,UI13,Senior Back End Developer,19.64,3.24
802,2017-11-17,00:09:58,Revenify,DevOps,4.57,call with XXX on demo preparation,UI36,Senior Full-Stack Developer,27.50,0.17
910,2017-11-28,00:47:49,WWU,Development,21.92,Values for this measure 80.1 that are presente...,UI36,Senior Full-Stack Developer,27.50,0.80
